<a href="https://colab.research.google.com/github/Corona-Locator-Nederland/corona-locator-nederland/blob/main/Corona_locator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from dotenv import load_dotenv, find_dotenv
try:
  load_dotenv(find_dotenv(), override=True)
except IOError:
  load_dotenv(find_dotenv(filename='dot.env'), override=True)

%run knack
if 'KNACK_APP_ID' in os.environ:
  knack = Knack(app_id = os.environ['KNACK_APP_ID'], api_key = os.environ['KNACK_API_KEY'])
else:
  knack = None

from ruamel.yaml import YAML
yaml = YAML(typ='safe')

from urllib.request import urlopen
from urllib.request import urlretrieve
import pandas as pd
import numpy as np
import json
import datetime
import requests
import json
import math
import os
import seaborn as sns
import glob
from dateutil.parser import parse as parsedate
import functools
import matplotlib.colors as colors

notebook True


Download de RIVM data als die nieuwer is dan wat we al hebben (gecached want de download van RIVM is *zeer* traag)

In [2]:
os.makedirs('downloads', exist_ok = True)

rivm = requests.head('https://data.rivm.nl/covid-19/COVID-19_casus_landelijk.csv')
latest = os.path.join('downloads', parsedate(rivm.headers['last-modified']).strftime('%Y-%m-%d@%H-%M.csv'))
if not os.path.exists(latest):
  print('downloading', latest)
  for f in glob.glob('downloads/*.csv'):
    if f != latest:
      os.remove(f)
  urlretrieve('https://data.rivm.nl/covid-19/COVID-19_casus_landelijk.csv', latest)
else:
  print(latest, 'exists')
covid = pd.read_csv(latest, sep=';', header=0 )
covid.head()

downloading downloads/2021-01-08@14-15.csv


,Date_file,Date_statistics,Date_statistics_type,Agegroup,Sex,Province,Hospital_admission,Deceased,Week_of_death,Municipal_health_service
0,2021-01-08 10:00:00,2020-01-01,DOO,40-49,Female,Noord-Holland,No,No,NaN,GGD Amsterdam
1,2021-01-08 10:00:00,2020-01-01,DOO,50-59,Male,Gelderland,No,No,NaN,Veiligheids- en Gezondheidsregio Gelderland-Mi...
2,2021-01-08 10:00:00,2020-01-01,DOO,20-29,Female,Zuid-Holland,No,No,NaN,GGD Hollands-Midden
3,2021-01-08 10:00:00,2020-01-02,DOO,70-79,Female,Noord-Holland,No,No,NaN,GGD Gooi en Vechtstreek
4,2021-01-08 10:00:00,2020-01-04,DOO,10-19,Female,Gelderland,Unknown,No,NaN,GGD Gelderland-Zuid


Download de bevolkings cijfers van CBS, uitgesplitst op de leeftijds categorien in de dataset van het RIVM

In [3]:
# https://www.cbs.nl/nl-nl/onze-diensten/open-data/open-data-v4/snelstartgids-odata-v4
def get_odata(target_url):
  data = pd.DataFrame()
  while target_url:
    r = requests.get(target_url).json()
    data = data.append(pd.DataFrame(r['value']))
        
    if '@odata.nextLink' in r:
      target_url = r['@odata.nextLink']
    else:
      target_url = None          
  return data

def roundup(x):
  return int(math.ceil(x / 10.0)) * 10
def rounddown(x):
  return int(math.floor(x / 10.0)) * 10

cbs = 'https://opendata.cbs.nl/ODataApi/OData/83482NED'

leeftijden = get_odata(cbs + "/Leeftijd?$select=Key, Title&$filter=CategoryGroupID eq 3")
leeftijden.set_index('Key', inplace=True)
# zet de Title om naar begin-eind paar
leeftijden_range = leeftijden['Title'].replace(r'^(\d+) tot (\d+) jaar$', r'\1-\2', regex=True).replace(r'^(\d+) jaar of ouder$', r'\1-1000', regex=True)
# splits die paren in van-tot
leeftijden_range = leeftijden_range.str.split('-', expand=True).astype(int)
# rond the "van" naar beneden op tientallen, "tot" naar boven op tientallen, en knip af naar "90+" om de ranges uit de covid tabel te matchen
leeftijden_range[0] = leeftijden_range[0].apply(lambda x: rounddown(x)).apply(lambda x: str(min(x, 90)))
leeftijden_range[1] = (leeftijden_range[1].apply(lambda x: roundup(x)) - 1).apply(lambda x: f'-{x}' if x < 90 else '+')
# en plak ze aan elkaar
leeftijden['Range'] = leeftijden_range[0] + leeftijden_range[1]
del leeftijden['Title']

def query(f):
  if f == 'Leeftijd':
    # alle leeftijds categerien zoals hierboven opgehaald
    return '(' + ' or '.join([f"{f} eq '{k}'" for k in leeftijden.index.values]) + ')'
  if f in ['Geslacht', 'Migratieachtergrond', 'Generatie']:
    # pak hier de key die overeenkomt met "totaal"
    ids = get_odata(cbs + '/' + f)
    return f + " eq '" + ids[ids['Title'].str.contains('totaal', na=False, case=False)]['Key'].values[0] + "'"
  if f == 'Perioden':
    # voor perioden pak de laatste
    periode = get_odata(cbs + '/Perioden').iloc[[-1]]['Key'].values[0]
    print('periode:', periode)
    return f"{f} eq '{periode}'"
  raise ValueError(f)
# haal alle properties op waar op kan worden gefiltered en stel de query samen. Als we niet alle termen expliciet benoemen is
# de default namelijk "alles"; dus als we "Geslacht" niet benoemen krijgen we de data voor *alle categorien* binnen geslacht.
filter = get_odata(cbs + '/DataProperties')
filter = ' and '.join([query(f) for f in filter[filter.Type != 'Topic']['Key'].values])

bevolking = get_odata(cbs + f"/TypedDataSet?$top=100&$filter={filter}&$select=Leeftijd, BevolkingOpDeEersteVanDeMaand_1")
# die _1 betekent waarschijnlijk dat het gedrag ooit gewijzigd is en er een nieuwe "versie" van die kolom is gepubliceerd
bevolking.rename(columns = {'BevolkingOpDeEersteVanDeMaand_1': 'BevolkingOpDeEersteVanDeMaand'}, inplace = True)
# merge de categoriecodes met de van-tot waarden
bevolking = bevolking.merge(leeftijden, left_on = 'Leeftijd', right_index = True)
# optellen om de leeftijds categorien bij elkaar te vegen zodat we de "agegroups" uit "covid" kunnen matchen 
bevolking = bevolking.groupby('Range')['BevolkingOpDeEersteVanDeMaand'].sum().to_frame()
# deze factor hebben we vaker nodig
bevolking['per 100k'] = 100000 / bevolking['BevolkingOpDeEersteVanDeMaand']
bevolking

periode: 2020MM12


,BevolkingOpDeEersteVanDeMaand,per 100k
Range,,
0-9,1759195,0.056844
10-19,1987617,0.050312
20-29,2240262,0.044638
30-39,2178284,0.045908
40-49,2169549,0.046093
50-59,2547659,0.039252
60-69,2139971,0.046730
70-79,1613006,0.061996
80-89,706728,0.141497


Bereken de stand van zaken van besmettingen / hospitalisaties / overlijden, per cohort in absolute aantallen en aantallen per 100k, met een kleur indicator voor de aantallen.

In [4]:
# vervang <50 en Unknown door Onbekend
covid['Cohort'] = covid['Agegroup'].replace({'<50': 'Onbekend', 'Unknown': 'Onbekend'})
# aangenomen 'gemiddelde' leeftijd van een cohort: minimum waarde + 5
assumed_cohort_age = [(cohort, [int(n) for n in cohort.replace('+', '').split('-')]) for cohort in covid['Cohort'].unique() if cohort[0].isdigit()]
assumed_cohort_age = { cohort: min(rng) + 5 for cohort, rng in assumed_cohort_age }
covid['Gemiddelde leeftijd'] = covid['Cohort'].apply(lambda x: assumed_cohort_age.get(x, np.nan))

# verwijder tijd
covid['Date_file_date'] = pd.to_datetime(covid['Date_file'].replace(r' .*', '', regex=True))

covid['Date_statistics_date'] = pd.to_datetime(covid['Date_statistics'])

# weken terug = verschil tussen Date_file en Date_statistcs, gedeeld door 7 dagen
covid['Weken terug'] = np.floor((covid['Date_file_date'] - covid['Date_statistics_date'])/np.timedelta64(7, 'D')).astype(np.int)

# voeg key, gem leeftijd, kleurnummer en totaal toe
Date_file = covid['Date_file_date'].unique()[0].astype('M8[D]').astype('O')
cohorten = list(bevolking.index) + ['Onbekend']
def summarize(df, category, prefix):
  # aangezien we hier de dataframe in-place wijzigen (bijv door toevoegen kolommen)
  # en we het 'covid' frame later nog clean nodig hebben
  df = df.copy(deep=True)
  
  df = (df
        .groupby(['Weken terug', 'Cohort'])['count']
        .sum()
        .unstack(fill_value=np.nan)
        .reset_index()
        .rename_axis(None, axis=1)
      ).merge(df
        # we voegen hier gemiddelde leeftijd toe, want die willen we op een ander
        # niveau aggregeren voor 'df' overschreven word
        .groupby(['Weken terug'])['Gemiddelde leeftijd']
        .mean()
        .to_frame(), on='Weken terug'
      )

  # altijd 52 rijen
  df = pd.Series(np.arange(52), name='Weken terug').to_frame().merge(df, how='left', on='Weken terug')

  # toevoegen missende cohorten
  for col in cohorten:
    if not col in df:
      df[col] = np.nan

  # sommeer per rij (axis=1) over de cohorten om een totaal te krijgen 
  df['Totaal'] = df[cohorten].sum(axis=1)

  # voeg periode en datum toe
  # periode afgeleid van weken-terug (= de index voor deze dataframe)
  df['Datum'] = pd.to_datetime(Date_file)
  df['Periode'] = (df
    .index.to_series()
    .apply(
      lambda x: (
        (Date_file + datetime.timedelta(weeks=-(x+1), days=1)).strftime('%d/%m')
        + ' - '
        + (Date_file + datetime.timedelta(weeks=-x)).strftime('%d/%m')
      )
    )
  )

  # voeg 'Key' en 'Type' kolom toe. Variabele 'type' kan niet, is een language primitive.
  df['Key'] = prefix + df.index.astype(str).str.rjust(3, fillchar='0')
  df['Type'] = category

  # voeg de kleur kolommen toe
  for col in cohorten:
    df['c' + col] = ((df[col] / df[[col for col in cohorten]].max(axis=1)) * 1000).fillna(0).astype(int)

  # herschikken van de kolommen
  colorder = ['Key', 'Weken terug', 'Datum', 'Periode', 'Gemiddelde leeftijd', 'Totaal', 'Type']
  return df[colorder + [col for col in df if col not in colorder]]

factor = bevolking.to_dict()['per 100k']
rivm = pd.concat(
  # flatten the result list zodat pd.concat ze onder elkaar kan plakken
  functools.reduce(lambda a, b: a + b, [
    [summarize(df.assign(count=1), label, prefix), summarize(df.assign(count=df['Cohort'].apply(lambda x: factor.get(x, np.nan))), label + ' per 100.000', prefix + '100k')]
    for df, label, prefix in [
      (covid, 'Positief getest', 'p'), # volledige count per cohort
      (covid[covid.Hospital_admission == 'Yes'], 'Ziekenhuisopname', 'h'), # count van cohort voor Hospital_admission == 'Yes'
      (covid[covid.Deceased == 'Yes'], 'Overleden', 'd'), # count van cohort voor Deceased == 'Yes'
    ]
  ])
)

# rood -> groen
cdict = {
  'red':   ((0.0, 0.0, 0.0),   # no red at 0
            (0.5, 1.0, 1.0),   # all channels set to 1.0 at 0.5 to create white
            (1.0, 0.8, 0.8)),  # set to 0.8 so its not too bright at 1
  'green': ((0.0, 0.8, 0.8),   # set to 0.8 so its not too bright at 0
            (0.5, 1.0, 1.0),   # all channels set to 1.0 at 0.5 to create white
            (1.0, 0.0, 0.0)),  # no green at 1
  'blue':  ((0.0, 0.0, 0.0),   # no blue at 0
            (0.5, 1.0, 1.0),   # all channels set to 1.0 at 0.5 to create white
            (1.0, 0.0, 0.0))   # no blue at 1
}
cm = colors.LinearSegmentedColormap('GnRd', cdict)
# geel -> paars
cm = sns.color_palette('viridis_r', as_cmap=True)
(rivm
  .fillna(0)
  .head()
  .round(1)
  .reset_index(drop=True)
  .style.background_gradient(cmap=cm, axis=1, subset=cohorten)
)

,Key,Weken terug,Datum,Periode,Gemiddelde leeftijd,Totaal,Type,0-9,10-19,20-29,30-39,40-49,50-59,60-69,70-79,80-89,90+,Onbekend,c0-9,c10-19,c20-29,c30-39,c40-49,c50-59,c60-69,c70-79,c80-89,c90+,cOnbekend
0,p000,0,2021-01-08 00:00:00,02/01 - 08/01,43.300000,28394.000000,Positief getest,416.000000,3803.000000,5881.000000,3576.000000,3724.000000,4542.000000,2724.000000,1788.000000,1420.000000,519.000000,1.000000,70,646,1000,608,633,772,463,304,241,88,0
1,p001,1,2021-01-08 00:00:00,26/12 - 01/01,45.200000,52057.000000,Positief getest,791.000000,5830.000000,8776.000000,6999.000000,7448.000000,9291.000000,5836.000000,3866.000000,2409.000000,810.000000,1.000000,85,627,944,753,801,1000,628,416,259,87,0
2,p002,2,2021-01-08 00:00:00,19/12 - 25/12,44.700000,66458.000000,Positief getest,1099.000000,8500.000000,10103.000000,8762.000000,10270.000000,11984.000000,7259.000000,4442.000000,3015.000000,1022.000000,2.000000,91,709,843,731,856,1000,605,370,251,85,0
3,p003,3,2021-01-08 00:00:00,12/12 - 18/12,43.300000,78019.000000,Positief getest,1269.000000,11906.000000,11818.000000,10232.000000,12350.000000,13426.000000,8129.000000,4821.000000,3049.000000,1013.000000,6.000000,94,886,880,762,919,1000,605,359,227,75,0
4,p004,4,2021-01-08 00:00:00,05/12 - 11/12,42.100000,62018.000000,Positief getest,1131.000000,11223.000000,8872.000000,8193.000000,9697.000000,10112.000000,6082.000000,3692.000000,2216.000000,797.000000,3.000000,100,1000,790,730,864,901,541,328,197,71,0


Publiceer de berekende statistieken indien we op github draaien


In [5]:
# publish
if 'GITHUB_TOKEN' in os.environ:
  os.makedirs('artifacts', exist_ok = True)
  print('Publishing to', os.environ['GITHUB_REPOSITORY'])
  today = os.path.join('artifacts', datetime.date.today().strftime('%Y-%m-%d') + '.csv')
  latest = 'artifacts/covid.csv'
  rivm.fillna(0).to_csv(latest, index=False)

  import github3 as github
  gh = github.GitHub(token=os.environ['GITHUB_TOKEN'], session=github.session.GitHubSession(default_read_timeout=60))
  repo = gh.repository(*os.environ['GITHUB_REPOSITORY'].split('/'))
  release = repo.release_from_tag('covid')
  assets = { asset.name: asset for asset in release.assets() }

  # remove existing
  for asset in [today, latest]:
    if os.path.basename(asset) in assets:
      assets[os.path.basename(asset)].delete()
    with open(latest) as f:
      release.upload_asset(asset=f, name=os.path.basename(asset), content_type='text/csv')

In [6]:
if knack:
  knack.update(scene='Leeftijdsgroepen', view='Leeftijdsgroepen', df=rivm.fillna(0).assign(Datum=rivm.Datum.dt.strftime('%Y-%m-%d')))

SyntaxError: positional argument follows keyword argument (<ipython-input-6-699f8c71ffe5>, line 2)